In [ ]:
BENCHMARK_TOOL = 'NVPROF' # or 'NCU'

import subprocess
import numpy as np
import pandas as pd
import io
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import LinearLocator, FormatStrFormatter, FuncFormatter
from matplotlib import cm 

In [ ]:
KERNELS_NAME = [
    "rgb2hsv", #0
    "rgb2hsvWithMinimumDivergence",
    "rgb2hsvWithCoordinatedOutputs",
    "histogram", #3
    "histogramWithSharedMemory",
    "histogramWithSharedMemoryAndHarcodedSize", #5
    "histogramWithMinimumDependencies",
    "repart", # 7
    "repartWithSharedMemory",
    "repartWithSharedMemoryAndHarcodedSize",
    "equalization", #10
    "equalizationWithConstantCoefficient",
    "hsv2rgb", #12
    "hsv2rgbWithMinimumDivergence" #13
]

KERNEL = {
    "rgb2hsv": [0, 1, 2],
    "hsv2rgb": [12, 13],
    "histogram": [3, 4, 5, 6],
    "repart": [7, 8, 9],
    "equalization" :[10, 11] 
} 
NUM_KERNEL = { # Numéro du kernel qu'il faut séléctionner pour lire 
    "rgb2hsv": 1,
    "histogram": 2, 
    "repart": 3,
    "equalization" : 4,
    "hsv2rgb": 5
} 

In [ ]:
def read_nvprof(output: str) -> pd.DataFrame:
    csv = output.split('\\n')
    columns = csv[3].split('\",\"')
    data = csv[4:]

    return pd.read_csv(io.StringIO('\n'.join(data)), names=columns)

    
def read_ncu(output: str) -> pd.DataFrame:
    csv = output.split('\\n')
    columns = csv[2].split('\",\"')
    data = csv[4:]

    return pd.read_csv(io.StringIO('\n'.join(data)), names=columns)

def benchmark_kernel(exe_path: str, *args) -> str :
    if BENCHMARK_TOOL == 'NVPROF':
        return str(subprocess.check_output(
            ['nvprof', '--print-gpu-trace', '--csv', exe_path, *args], 
            stderr=subprocess.STDOUT
        ))
    elif BENCHMARK_TOOL == 'NCU':
        return str(subprocess.check_output(
            ['ncu','--csv', exe_path, *args], 
            stderr=subprocess.STDOUT, shell=True
        ))

def read_benchmark(output: str) -> pd.DataFrame:
    if BENCHMARK_TOOL == 'NVPROF':
        return read_nvprof(output)
    elif BENCHMARK_TOOL == 'NCU':
        return read_ncu(output)

def get_kernel_duration(df: pd.DataFrame, kernelNum=0) -> float:
    if BENCHMARK_TOOL == 'NVPROF':
        # conversion en ms 
        if df.loc[0, 'Duration'] == "ms":
            return float(df.loc[kernelNum, 'Duration']) 
        if df.loc[0, 'Duration'] == "s":
            return float(df.loc[kernelNum, 'Duration']) * 1e3
        # probablement en micro seconde
        return float(df.loc[kernelNum, 'Duration']) * 1e-3
    elif BENCHMARK_TOOL == 'NCU':
        return float(df[df['Metric Name'] == 'Duration']['Metric Value'].to_numpy().item().replace(',','')) * 1e-6

In [ ]:
# Renvoie les durée moyennes en fonction de variation de tailles de l'axe X des blocs pour un certain kernel
def evaluateBlocSize(kernelNum, kernelInd, nTry = 10):  
    durations = []
    sizes = []
    avg = 0
    xBlockSize, yBlockSize = 1, 1
    
    while xBlockSize <= 1024:
        for _ in range(nTry) :         
            avg += get_kernel_duration(read_benchmark(
                benchmark_kernel("./hist", 
                    "-f", "./img/chateau.png", 
                    "-b", str(kernelInd), 
                    "-d", str(xBlockSize), str(yBlockSize)
                )
            ), kernelNum)

        avg /= nTry
        durations.append(avg)    
        sizes.append(xBlockSize)
        xBlockSize *= 2
    return durations, sizes

In [ ]:
def showHistogramDuration(durations, sizes, title, xLabel, yLabel):
    plt.title(title)  
    plt.xlabel(xLabel) 
    plt.ylabel(yLabel) 
    plt.bar(np.array(sizes).astype(str), np.array(durations), width=.8) 
    plt.show()

def showHistogramOfKernels(kernels, nTry = 10):  
    global NUM_KERNEL
    i = 0
    for k in kernels: 
        durations, sizes = evaluateBlocSize(NUM_KERNEL[i], i, nTry)
        showHistogramDuration(durations, sizes, k, "taille de bloc", "ms") 
        i+=1
#print("Évaluation des kernels suivant la variation de la taille des blocs sur l'axe X") 
#showHistogramOfKernels(KERNELS, 5)

In [ ]:
def evaluate2DBlocSize(kernelName, nTry = 10):  
    global KERNELS_NAME
    durations = []
    sizes = [] 
    xBlockSize = 1
    step = 64
    kernelNum = KERNELS_NAME.index(kernelName)

    while xBlockSize <= 1024:
        yBlockSize = 1
        durations.append([])
        while yBlockSize <= 1024:
            if (xBlockSize * yBlockSize) > 1024:
                durations[-1].append(np.nan)     
                yBlockSize *= 2
                continue
            
            avg = 0 
            for _ in range(nTry) :      
                output = read_benchmark(
                    benchmark_kernel("./hist", 
                        "-f", "./img/chateau.png", 
                        "-b", str(kernelNum), 
                        "-d", str(1), str(1)
                    )
                )     
                cond = output["Name"].str.contains(kernelName, na=False, case=False) 
                i = output.index[cond][0] # Trouve le premier indice  
                avg += get_kernel_duration(output, i) 

            avg /= nTry
            durations[-1].append(avg)    
            yBlockSize *= 2
        sizes.append(xBlockSize)
          
        xBlockSize *= 2
    return durations, sizes
 

In [ ]:
#print("Variation de la taille des blocs rgb to hsv")
#durations, sizes = evaluate2DBlocSize(1, 1)   

In [ ]:
def getBestParameters(durations, sizesX, sizesY):
    m = durations[0][0]
    x = 1
    y = 1

    for i in range(len(durations)):
        for j in range(len(durations[i])):  
            if (durations[i][j] < m and not np.isnan(durations[i][j])):
                m = durations[i][j]
                x = sizesX[i] 
                y = sizesY[j] 

    return m, x, y 

def log_tick_formatter(val, pos=None): 
    return int(10**val) 

def showHistogramDuration3DHist(durations, sizesX, sizesY, title, xLabel, yLabel): 
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.set_title("Temps d'éxecution du kernel: " + title)
 
    xpos, ypos = np.meshgrid(sizesX, sizesY)
    xpos = xpos.flatten('F')
    ypos = ypos.flatten('F') 
    dz = np.array(durations).flatten()  
    minZ, maxZ = min(dz), max(dz)
    zpos = np.full_like(dz, minZ, dtype=np.double)  
    cmap = cm.get_cmap('jet')   
    rgb = [cmap((k-minZ)/ (maxZ-minZ)) for k in dz] 
    dz = np.array([d - minZ for d in dz])
    dx = 0.5 * np.ones_like(zpos)
    dy = dx.copy()   
    hist = ax.bar3d(np.log10(xpos), np.log10(ypos), zpos, 0.13, 0.13, dz, color=rgb, alpha=0.3, zsort='average')

    ##### Parametrage des axes #####
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
    # Affichage de l'echelle logarithmic
    ax.xaxis.set_major_formatter(FuncFormatter(log_tick_formatter))
    ax.yaxis.set_major_formatter(FuncFormatter(log_tick_formatter)) 
  
    #ax.set_zlim(minZ /2, maxZ * 2)
    ax.set_zlim(minZ, maxZ)
    ax.set_xlabel(xLabel)
    ax.set_ylabel(yLabel) 
    ax.set_zlabel("durée (en ms)")  
    
    # Affichage des meilleurs paramêtres 
    d, x, y  = getBestParameters(durations, sizesX, sizesY) 
    plt.figtext(0.53, -0.03, "Meilleur temps: " + str(minZ) + " ms\navec " + xLabel + " : " + str(x) + "\n et " + yLabel + " : " + str(y) , ha="center", bbox={"facecolor":"white", "alpha":0.5, "pad":5})
    plt.show()
    #plt.savefig(title)

# showHistogramDuration3DHist(durations, sizes, sizes, KERNELS_NAME[0], "taille de bloc x", "taille de bloc y")  
#durations, sizes = evaluate2DBlocSize(NUM_KERNEL["rgb2hsv"], 2, 1)   
#print(durations, sizes)
  

In [ ]:
def testBlocKernels(kernelNameType, nTry = 1):
    global KERNEL, KERNELS_NAME, NUM_KERNEL
    
    for i in KERNEL[kernelNameType]:
        k = KERNELS_NAME[i]
        print("Execution du kernel : ", k, "avec ", nTry, " essaie.")
        durations, sizes = evaluate2DBlocSize(k, nTry)    
        showHistogramDuration3DHist(durations, sizes, sizes, k, "taille de bloc x", "taille de bloc y")  
#showHistogramDuration3DHist(durations, sizes, sizes, KERNELS_NAME[0], "taille de bloc x", "taille de bloc y")
#testBlocKernels([2], 1) # avec un seul essaie, on remarque que les temps sont imprécies et trop hétérogènes, il faut prendre le temps moyen entre plusieurs execution du même kernel

Test de l'affichage de chaque kernel

In [ ]:
#testBlocKernels("rgb2hsv", 1)
k = "rgb2hsvWithMinimumDivergence"
durations, sizes = evaluate2DBlocSize(k, 1)    

showHistogramDuration3DHist(durations, sizes, sizes, k, "taille de bloc x", "taille de bloc y")  

# Evaluation des meilleurs dimensions 2D de bloc 
La taille de la grille sera égale à 1. 

In [ ]:
nTry = 10
testBlocKernels("rgb2hsv", nTry) 

In [ ]:
testBlocKernels("hsv2rgb", nTry) 

In [ ]:
testBlocKernels("histogram", nTry)

In [ ]:
testBlocKernels("repart", nTry)

In [ ]:
testBlocKernels("equalization", nTry)

# Variation des taille de bloc et de grille en 1D

In [ ]:
def evaluate1DBlocAndGridSize(kernelName, nTry = 10): # On test les variations de taille de bloc et de grille sur une seul dimension 
    global KERNELS_NAME
    durations = []
    sizes = [] 
    blockSize = 1 
    kernelNum = KERNELS_NAME.index(kernelName)

    while blockSize <= 1024:
        gridSize = 1
        durations.append([])
        while gridSize <= 1024:
           # if (blockSize * gridSize) > 1024:
          #      durations[-1].append(np.nan)     
          #      gridSize *= 2
          #      continue
            
            avg = 0 
            for _ in range(nTry) :     
                output = read_benchmark(
                    benchmark_kernel("./hist", 
                        "-f", "./img/chateau.png", 
                        "-b", str(kernelNum), 
                        "-d", str(blockSize), "1",
                        "-D", str(gridSize), "1"
                    )
                )     
                cond = output["Name"].str.contains(kernelName, na=False, case=False) 
                i = output.index[cond][0] # Trouve le premier indice  
                avg += get_kernel_duration(output, i) 

            avg /= nTry
            durations[-1].append(avg)    
            gridSize *= 2
        sizes.append(blockSize)
          
        blockSize *= 2
    return durations, sizes
    
def testBlocAndGridKernels(kernel, i, nTry = 1):
    global KERNEL_NAME, KERNEL, NUM_KERNEL 
    print("Execution du kernel : ", KERNELS_NAME[i], "avec ", nTry, " essaie.")
    durations, sizes = evaluate1DBlocAndGridSize(NUM_KERNEL[kernel], i, nTry)   
    showHistogramDuration3DHist(durations, sizes, sizes, KERNELS_NAME[i], "taille de bloc", "taille de grille")  

In [ ]:
nTry = 10
testBlocAndGridKernels("rgb2hsv", KERNEL["rgb2hsv"][2], nTry) 

In [ ]:
testBlocAndGridKernels("histogram", KERNEL["histogram"][0], nTry)

In [ ]:
testBlocAndGridKernels("repart", KERNEL["repart"][0], nTry)

In [ ]:
testBlocAndGridKernels("equalization", KERNEL["equalization"][1], nTry) #

In [ ]:
testBlocAndGridKernels("hsv2rgb", KERNEL["hsv2rgb"][1], nTry)